In [94]:
import sys
sys.path.append('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks')

In [95]:
from brokers.sim import Sim
from brokers.ibkr import IBKR
import pandas as pd
from datetime import datetime
import numpy as np

In [96]:
sim = Sim()
ibkr = IBKR()

In [97]:
import os

files_list = os.listdir('/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/ibkr/1d/')
symbols_loaded = []
for filename in files_list:
    symbol = filename.split('.')[0]
    if '/' not in symbol and len(symbol) > 0:
        symbols_loaded.append(symbol)

stock_symbols = symbols_loaded[:20]
symbols_to_remove = ['BRK/A', 'BRK/B', 'MNST', 'LIN', 'TPL']
for symbol_to_remove in symbols_to_remove:
    if symbol_to_remove in stock_symbols:
        stock_symbols.remove(symbol_to_remove)
        
interval_inputs = {'1d': {'columns': ['open', 'high', 'close', 'low'], 'lookback': 370}}
start_date = pd.Timestamp(datetime(2005,3,15)).tz_localize('UTC').tz_convert('EST')
end_date = pd.Timestamp(datetime(2011,2,6)).tz_localize('UTC').tz_convert('EST')
lookback = {'1d': 100}

In [98]:
combined_df = ibkr.data.update_price_data(stock_symbols, interval_inputs, data_folder='/Users/vandanchopra/Vandan_Personal_Folder/CODE_STUFF/Projects/mathematricks/db/data/ibkr', throttle_secs=0.25, start_date=start_date, end_date=end_date, lookback=lookback)

Fetching 1d CSV data & Bifurcating Batches: 100%|██████████| 20/20 [00:00<00:00, 81.49it/s]
2024-12-20 11:14:09,499 - IBKR-data - INFO - {'interval': '1d', 'stock_symbols_no_data': 0, 'stock_symbols_with_partial_data': 0, 'stock_symbols_with_full_data': 20} - ibkr.py:374
Get the data for the ones that have full data: 100%|██████████| 20/20 [00:00<00:00, 1542.70it/s]


In [99]:
combined_df.loc['1d']['close']

symbol,AIRT,ANTE,CSCO,EFC,ISRG,ISSC,NVR,PRI,UFCS,WOR
date,,,,,,,,,,
2004-12-03 23:59:59+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67.96,NaN
2004-12-04 23:59:59+00:00,15.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,12.95
2004-12-07 23:59:59+00:00,15.82,NaN,NaN,NaN,NaN,31.69,NaN,NaN,67.85,12.77
2004-12-08 23:59:59+00:00,14.31,NaN,NaN,NaN,NaN,30.9,NaN,NaN,67.12,12.58
2004-12-09 23:59:59+00:00,14.7,NaN,NaN,NaN,NaN,30.72,NaN,NaN,67.99,12.37
...,...,...,...,...,...,...,...,...,...,...
2011-02-01 23:59:59+00:00,6.54,135.8,20.73,22.21,36.29,5.79,765.0,24.2,20.05,11.71
2011-02-02 23:59:59+00:00,6.59,144.8,NaN,22.2,NaN,5.97,771.56,24.69,20.52,12.06
2011-02-03 23:59:59+00:00,6.53,143.6,NaN,22.0,NaN,5.98,767.93,24.33,19.95,12.03


# DEVELOPMENT

In [100]:
from copy import deepcopy

market_data_df_orig = deepcopy(combined_df)
market_data_df_orig.ffill(inplace=True)
market_data_df_orig = market_data_df_orig.infer_objects(copy=False)

/var/folders/d0/_rd9vn610y36nmcvbvp1xr480000gn/T/ipykernel_52691/3870097132.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  market_data_df_orig.ffill(inplace=True)


In [101]:
long_short_symbol_count = 10
shortlen=1000
longlen = 2 * shortlen
basisLen=12
offset=0.85
offsetSigma=3
granularity = '1d'
df = deepcopy(market_data_df_orig)
open_symbols = []


In [103]:
df_1d = df.loc[granularity]

In [111]:
def calculate_bounce_strength(df_1d):
    days_ago_list_difference = 15
    days_ago_list = list(range(1, 365, days_ago_list_difference))
    strategy_df_orig = df_1d[::-1].iloc[days_ago_list]['close']
    # Drop all assets with NaN values
    strategy_df_orig = strategy_df_orig.dropna(axis=1)
    strategy_df = strategy_df_orig.copy()
    # Create a list from days_ago_list that has each element followed by '_days_ago'
    price_days_ago_list_str = [str(x) + '_days_ago_price' for x in days_ago_list]
    # strategy_df['datetime'] = strategy_df.index
    # replace the index with the days_ago_list
    strategy_df.index = price_days_ago_list_str

    '''STEP 3: Calculate growth rate 1 on strategy dataframe: Growth Rate Period Based'''
    # Calculate the growth rate for each row
    growth_rate_pct_diff_df = (strategy_df - strategy_df.shift(-1))/strategy_df.shift(-1)
    growth_rate_pct_diff_df = growth_rate_pct_diff_df.dropna()
    growth_rate_pct_diff_days_ago_list_str = [str(x) + '_days_ago_growth_1' for x in days_ago_list[1:]]
    growth_rate_pct_diff_df.index = growth_rate_pct_diff_days_ago_list_str
    # Normalize growth_rate_1_df horizontally like this (x - min(x))/(max(x) - min(x))
    growth_rate_pct_diff_df_normalized_scores = growth_rate_pct_diff_df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis=1)

    '''STEP 4: Calculate growth rate 2 on strategy dataframe. Which is everything divided by the first row: Growth Rate Cumulative Based'''
    # Calculate the growth rate for each row
    growth_rate_full_df = (strategy_df.iloc[0]-strategy_df)/strategy_df
    growth_rate_full_days_ago_list_str = [str(x) + '_days_ago_growth_2' for x in days_ago_list[1:]]
    growth_rate_full_df = growth_rate_full_df.drop(growth_rate_full_df.index[0])
    growth_rate_full_df.index = growth_rate_full_days_ago_list_str

    # Normalize growth_rate_1_df horizontally like this (x - min(x))/(max(x) - min(x))
    growth_rate_full_df_normalized_scores = growth_rate_full_df.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)), axis=1)

    '''STEP 5: Get the Weights for each row'''
    def calculate_u_curve_weights(points_count, mode='normal'):
            x = np.linspace(-20, 10, points_count)
            
            # Define a quadratic function that is high at the edges and lower in the middle
            arr = ((-0.5 * x**2) + 4) * -1
            # Ensure all values are positive
            if np.min(arr) < 0:
                arr = arr + abs(np.min(arr)-4)

            # Normalize the array so the sum is 100
            arr = (arr / np.sum(arr)) * 100
            
            return arr
    weights_1 = calculate_u_curve_weights(len(growth_rate_pct_diff_df_normalized_scores))
    weights_2 = calculate_u_curve_weights(len(growth_rate_full_df_normalized_scores))

    '''Step 6: Calculate the weight score for each row'''
    scores_1_df = growth_rate_pct_diff_df_normalized_scores.multiply(weights_1, axis=0)
    scores_2_df = growth_rate_full_df_normalized_scores.multiply(weights_2, axis=0)

    '''STEP 7: Combine the scores and get the final Score'''
    # Concatenate the scores_1_df and scores_2_df
    weights_for_scores_combination = [10, 10]
    scores_df = pd.concat([scores_1_df * weights_for_scores_combination[0], scores_2_df * weights_for_scores_combination[1]])
    final_scores = pd.DataFrame(scores_df.sum(axis=0))
    # give the columns names
    final_scores.columns = ['final_score']
    final_scores = final_scores.sort_values(by='final_score', ascending=False)
    return final_scores

In [122]:
short_sma = df_1d['close'].rolling(window=shortlen, min_periods=1, center=False).mean().iloc[-1]
long_sma = df_1d['close'].rolling(window=longlen, min_periods=1, center=False).mean().iloc[-1]

trend_strength = short_sma/long_sma
sma_strength_long_symbols = trend_strength[(trend_strength > 1) & (trend_strength > 1)]
sma_strength_short_symbols = trend_strength[(trend_strength < 1) & (trend_strength < 1)]

# Sort long_signals in descending order
sma_strength_long_symbols = sma_strength_long_symbols.sort_values(ascending=False)
sma_strength_short_symbols = sma_strength_short_symbols.sort_values(ascending=True)

# Trim the list of signals
max_required_len = long_short_symbol_count * 2
sma_strength_long_symbols = sma_strength_long_symbols[:max_required_len]
sma_strength_short_symbols = sma_strength_short_symbols[:max_required_len]

tickers_temp = list(set(list(sma_strength_long_symbols.index) + list(sma_strength_short_symbols.index) + open_symbols))
if len(tickers_temp) == 0:
    tickers_temp = [df_1d['close'].columns[0]]
trimmed_input_df = df.loc['1d', pd.IndexSlice[:, tickers_temp]]
df_1d_trimmed = trimmed_input_df.copy()

short_sma_trimmed = short_sma[short_sma.index.isin(tickers_temp)]
long_sma_trimmed = long_sma[long_sma.index.isin(tickers_temp)]

def computed_prices(df, length=9, offset=0.85, sigma=6):
    if length < 1:
        return df

    # Precompute Gaussian weights
    m = offset * (length - 1)
    s = length / sigma
    idx = np.arange(length)
    w = np.exp(-((idx - m) ** 2) / (2 * s * s))
    w /= w.sum()

    # Perform weighted rolling computation across all columns (vectorized)
    def rolling_weighted_sum(x):
        # Dynamically adjust weights to match the size of x
        current_len = len(x)
        w_adjusted = w[:current_len] / w[:current_len].sum()  # Normalize weights for the current window
        return np.dot(x, w_adjusted)

    # Apply rolling weighted sum across all symbols at once
    return df.rolling(window=length, min_periods=1).apply(rolling_weighted_sum, raw=True)
closeSeries = computed_prices(df_1d_trimmed['close'], length=basisLen, offset=offset, sigma=offsetSigma).iloc[-1]
openSeries = computed_prices(df_1d_trimmed['open'], length=basisLen, offset=offset, sigma=offsetSigma).iloc[-1]
closeSeries = closeSeries.loc[tickers_temp]
openSeries = openSeries.loc[tickers_temp]

# Create Positions
# conditions = [
#     ((closeSeries.gt(openSeries)) & (short_sma_trimmed.gt(long_sma_trimmed))),
#     ((closeSeries.lt(openSeries)) & (short_sma_trimmed.lt(long_sma_trimmed)))
# ]
# choices = [1, -1]
# positions = np.select(conditions, choices, default=0)
decision_matrix_df = pd.DataFrame(index=tickers_temp)
# Include symbol LTP        
symbol_ltp = df_1d_trimmed.iloc[-1]['close']
decision_matrix_df['symbol_ltp'] = symbol_ltp.loc[symbol_ltp.index.isin(decision_matrix_df.index)]

decision_matrix_df['bounce_strength'] = 0
bounce_strength = calculate_bounce_strength(df_1d_trimmed)
bounce_strength = bounce_strength/bounce_strength.mean()
decision_matrix_df['bounce_strength'] = bounce_strength.loc[bounce_strength.index.isin(decision_matrix_df.index)]
decision_matrix_df['bouce_strength_direction'] = 0
decision_matrix_df.loc[decision_matrix_df['bounce_strength'] > 1, 'bouce_strength_direction'] = 1
decision_matrix_df.loc[decision_matrix_df['bounce_strength'] < 1, 'bouce_strength_direction'] = -1

decision_matrix_df['short_sma_trimmed'] = short_sma_trimmed
decision_matrix_df['long_sma_trimmed'] = long_sma_trimmed
decision_matrix_df['trend_strength'] = trend_strength.loc[trend_strength.index.isin(decision_matrix_df.index)]

decision_matrix_df['trend_strength_direction'] = 0
decision_matrix_df.loc[(decision_matrix_df['trend_strength'] > 1) & (decision_matrix_df['trend_strength'] > 1.04), 'trend_strength_direction'] = 1
decision_matrix_df.loc[(decision_matrix_df['trend_strength'] < 1) & (decision_matrix_df['trend_strength'] < 0.96), 'trend_strength_direction'] = -1

# self.logger.debug({'decision_matrix_df':decision_matrix_df})
decision_matrix_df['trend_strength_score'] = float(0)
decision_matrix_df.loc[(decision_matrix_df['trend_strength'] > 1) & (decision_matrix_df['trend_strength'] != 0), 'trend_strength_score'] = decision_matrix_df['trend_strength'].astype(float)
decision_matrix_df.loc[(decision_matrix_df['trend_strength'] < 1) & (decision_matrix_df['trend_strength'] != 0), 'trend_strength_score'] = (1/decision_matrix_df['trend_strength']).astype(float)

# Include closeSeries and openSeries
decision_matrix_df['closeSeries'] = closeSeries.loc[closeSeries.index.isin(decision_matrix_df.index)]
decision_matrix_df['openSeries'] = openSeries.loc[openSeries.index.isin(decision_matrix_df.index)]

decision_matrix_df['entry_indicator_direction'] = decision_matrix_df['closeSeries'] / decision_matrix_df['openSeries']
decision_matrix_df['entry_strength_indicator_direction'] = 0
decision_matrix_df.loc[decision_matrix_df['closeSeries'] > decision_matrix_df['openSeries'], 'entry_strength_indicator_direction'] = 1
decision_matrix_df.loc[decision_matrix_df['closeSeries'] < decision_matrix_df['openSeries'], 'entry_strength_indicator_direction'] = -1

decision_matrix_df['entry_indicator_strength'] = decision_matrix_df['closeSeries'] / decision_matrix_df['openSeries']
# if entry_indicator_strength < 1, then make it 1/entry_indicator_strength
decision_matrix_df.loc[decision_matrix_df['entry_indicator_strength'] < 1, 'entry_indicator_strength'] = 1/decision_matrix_df['entry_indicator_strength']

decision_matrix_df['position'] = 0

decision_matrix_df.loc[(decision_matrix_df['bouce_strength_direction'] == 1) & (decision_matrix_df['trend_strength_direction'] == 1) & (decision_matrix_df['entry_strength_indicator_direction'] == 1), 'position'] = 1
decision_matrix_df.loc[(decision_matrix_df['bouce_strength_direction'] == -1) & (decision_matrix_df['trend_strength_direction'] == -1) & (decision_matrix_df['entry_strength_indicator_direction'] == -1), 'position'] = -1

decision_matrix_df['position_score'] = decision_matrix_df['position'] * decision_matrix_df['trend_strength_score']

# sort by position_score
decision_matrix_df = decision_matrix_df.sort_values(by='position_score', ascending=False)

In [123]:
decision_matrix_df

,symbol_ltp,bounce_strength,bouce_strength_direction,short_sma_trimmed,long_sma_trimmed,trend_strength,trend_strength_direction,trend_strength_score,closeSeries,openSeries,entry_indicator_direction,entry_strength_indicator_direction,entry_indicator_strength,position,position_score
ISRG,36.77,1.353259,1,27.46715,22.247320,1.234627,1,1.234627,35.054517,34.684240,1.010676,1,1.010676,1,1.234627
WOR,12.11,1.251174,1,8.97282,10.507205,0.853968,-1,1.171004,11.939982,12.000510,0.994956,-1,1.005069,0,0.000000
ANTE,140.00,1.132028,1,142.66607,179.329881,0.795551,-1,1.256990,140.460246,141.732916,0.991021,-1,1.009061,0,0.000000
NVR,760.61,1.099376,1,594.42372,618.304445,0.961377,0,1.040175,771.911756,772.475105,0.999271,-1,1.000730,0,0.000000
ISSC,5.95,1.041751,1,5.16158,10.675140,0.483514,-1,2.068192,5.934574,5.924111,1.001766,1,1.001766,0,0.000000
AIRT,6.64,0.796558,-1,6.13918,6.378150,0.962533,0,1.038925,6.555654,6.576103,0.996890,-1,1.003119,0,0.000000
CSCO,21.54,0.697226,-1,21.56584,22.692475,0.950352,-1,1.052242,20.981798,20.995907,0.999328,-1,1.000672,-1,-1.052242
UFCS,20.15,0.628628,-1,21.65040,28.657360,0.755492,-1,1.323641,20.246374,20.366728,0.994091,-1,1.005944,-1,-1.323641


In [ ]:
decision_matrix_df

In [ ]:
decision_matrix_df['market_regime'] = 0
count = 

In [ ]:
decision_matrix_df[decision_matrix_df['position_score'] > 0].shape[0], decision_matrix_df[decision_matrix_df['position_score'] < 0].shape[0], decision_matrix_df[decision_matrix_df['position_score'] == 0].shape[0]

In [ ]:
# Create signal Strength based on position_score, the max signal_strength is 10 and the minimum is 0
decision_matrix_df['signal_strength'] = 0
decision_matrix_df.loc[decision_matrix_df['position_score'] > 0, 'signal_strength'] = 10
decision_matrix_df.loc[decision_matrix_df['position_score'] < 0, 'signal_strength'] = 0
